In [6]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:98% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:21pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html>pre{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

<b><font color="red" size="6">ch14. 웹데이터 수집</font></b>
# 1절. BeautifulSoup과 parser
```pip install bs4``` 아나콘다를 설치하면 7500개여개의 패키지 설치
- 공식 사이트 : https://www.crummy.com/software/BeautifulSoup/
- Docs : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [9]:
import requests # HTTP 요청 처리 lib
from requests_file import FileAdapter

In [117]:
s = requests.Session() # HTTP 요청관리를 위한 세션 객체
s.mount("file://", FileAdapter())
response = s.get('file:///ai_x/lecNote/01_python/data/ch14_sample.html')
response

<Response [200]>

In [6]:
response.status_code 
# 200 : 정상
# 404 : 없는 페이지
# 406 : get, post 오류

200

In [9]:
response.content # 바이너리 형식의 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [10]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [11]:
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [118]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,  # response.text
                    "html.parser")
# soup

In [31]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 title속성 :', el.attrs['title'])
print('el의 title속성 :', el.attrs.get('title') )
print('el의 이름 :', el.name)

el : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}
el의 title속성 : greeting
el의 title속성 : greeting
el의 이름 : h1


In [38]:
# soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로
el = soup.select('h1')
print('리스트 el :', el)
print('el의 text들 :', [e.text for e in el])
# for e in el:
#     print(e.text, end=',')
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs.get('class') for e in el])

리스트 el : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el의 text들 : ['Hello, CSS', 'Hi, CSS']
el의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}, {'class': ['css']}]
el의 class속성들 : [['greeting', 'css'], ['css']]


In [119]:
# select_one(선택자)와 find(태그, 속성)
print('select_one :', soup.select_one('h1.css'))
print('find :', soup.find('h1', {'class':'css'}))
print('find :', soup.find('h1', class_='css'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1', {'id':'text'}));

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>


In [120]:
# select(선택자)와 find_all(태그, 속성)
print('모든 h1.css, span 태그(select) :',
     soup.select('h1.css, span'))
print('모든 h1.css, span태그(find_all) :',
     soup.find_all(['h1','span'], class_='css'))

모든 h1.css, span 태그(select) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모든 h1.css, span태그(find_all) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [57]:
# 없는 엘리먼트 찾기
print('find_all(빈list) :', soup.find_all('a', class_='css'))
print('find(None) :', soup.find('a', class_='css'))
print('select(빈list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))

find_all(빈list) : []
find(None) : None
select(빈list) : []
select_one(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
- json, html, xml
## 2.1 JSON파일
- request모듈(get)
- urllib.request모듈(urlopen)

In [ ]:
# 크롤링 허용 범위는 사이트마다 ~/robots.txt에서 확인할 수 있습니다

In [12]:
# 방법1
import requests
response = requests.get('http://api.github.com')
response, response.status_code

(<Response [200]>, 200)

In [11]:
# 방법2
from urllib.request import urlopen
response = urlopen('http://api.github.com')
response

In [125]:
text = '{"속성1": "값1","속성2":"값2"}'
type(text)

str

In [126]:
# 문자(딕셔너리 타입)를 딕셔너리
# '{"속성1": "값1","속성2":"값2"}' => {"속성1": "값1","속성2":"값2"}
import json
json.loads(text)

{'속성1': '값1', '속성2': '값2'}

## 2.2 html 파일
### 1) 환율정보 가져오기 (네이버->증권->시장지표)
- https://finance.naver.com/marketindex/

In [68]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex'
response = requests.get(url)
# response, response.status_code
# response.content 
# response.content.decode('cp949') == response.text
soup = BeautifulSoup(response.text, # response.content
                    'html.parser')

In [78]:
# 방법2
from urllib.request import urlopen
response = urlopen(url)
# response.status 상태코드
# response.read()
# response.read().decode('cp949')
soup = BeautifulSoup(response,  # response.read()
                     'html.parser')

In [80]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [97]:
price = soup.select('div.head_info > span.value')
[p.text for p in price]
[round(float(p.text.replace(',',''))) for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

[1375, 952, 1556, 191, 144, 1, 1, 99, 61, 1633, 3300, 146168]

In [87]:
out = '1,374.70'
round(float(''.join(out.split(','))))

1375

In [90]:
round(float(out.replace(',','')))

1375

In [100]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7, '') # 7번째 index에 '' 추가
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [105]:
status = soup.select('div.head_info > span.blind')
[t.text for t in title]
[p.text for p in price]
unit
[s.string for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [106]:
len(title), len(price), len(unit), len(status)

(12, 12, 12, 12)

In [111]:
for idx in range(len(title)):
    print("{}. {} : {}{} - {}".format(idx+1,
                                     title[idx].text,
                                     price[idx].text,
                                     unit[idx],
                                     status[idx].text))

1. 미국 USD : 1,374.70원 - 하락
2. 일본 JPY(100엔) : 952.47원 - 하락
3. 유럽연합 EUR : 1,555.89원 - 하락
4. 중국 CNY : 190.99원 - 하락
5. 달러/일본 엔 : 144.3800엔 - 상승
6. 유로/달러 : 1.1342달러 - 하락
7. 영국 파운드/달러 : 1.3520달러 - 하락
8. 달러인덱스 : 99.4200 - 상승
9. WTI : 60.89달러 - 하락
10. 휘발유 : 1633.17원 - 하락
11. 국제 금 : 3300.4달러 - 하락
12. 국내 금 : 146167.95원 - 상승


In [112]:
for t, p, u, s  in zip(title, price, unit, status):
    print("{} : {}{} - {}".format(t.text,
                                 p.text,
                                 u,
                                 s.text))

미국 USD : 1,374.70원 - 하락
일본 JPY(100엔) : 952.47원 - 하락
유럽연합 EUR : 1,555.89원 - 하락
중국 CNY : 190.99원 - 하락
달러/일본 엔 : 144.3800엔 - 상승
유로/달러 : 1.1342달러 - 하락
영국 파운드/달러 : 1.3520달러 - 하락
달러인덱스 : 99.4200 - 상승
WTI : 60.89달러 - 하락
휘발유 : 1633.17원 - 하락
국제 금 : 3300.4달러 - 하락
국내 금 : 146167.95원 - 상승


In [115]:
for idx, (t, p, u, s)  in enumerate(zip(title, price, unit, status)):
    print("{}. {} : {}{} - {}".format(idx+1,
                                t.text,
                                p.text,
                                u,
                                s.text))

1. 미국 USD : 1,374.70원 - 하락
2. 일본 JPY(100엔) : 952.47원 - 하락
3. 유럽연합 EUR : 1,555.89원 - 하락
4. 중국 CNY : 190.99원 - 하락
5. 달러/일본 엔 : 144.3800엔 - 상승
6. 유로/달러 : 1.1342달러 - 하락
7. 영국 파운드/달러 : 1.3520달러 - 하락
8. 달러인덱스 : 99.4200 - 상승
9. WTI : 60.89달러 - 하락
10. 휘발유 : 1633.17원 - 하락
11. 국제 금 : 3300.4달러 - 하락
12. 국내 금 : 146167.95원 - 상승


### 2) 이번주 로또번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin(google에 "로또번호 당첨번호" 검색)
```
    1173회(2025년 05월 24일 추첨)
    당첨번호 [1 5 18 20 30 35]
    보너스 3
```

In [13]:
# 방법1의 soup객체 생성
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")

In [14]:
# 방법2의 sou객체 생성
from urllib.request import urlopen
response = urlopen(url)
soup = BeautifulSoup(response, "html.parser")

In [19]:
times = soup.select_one('div.win_result > h4 > strong').text # 1173회
date = soup.select_one('div.win_result >p.desc').text  #(2025년 05월 24일 추첨)
print(times, date)
title1 = soup.select_one('div.num.win > strong').text
lotto_number = soup.select('div.num.win > p > span')
print(title1, [int(lotto.text) for lotto in lotto_number])
title2 = soup.select_one('div.num.bonus > strong').string
bonus_number = soup.select_one('div.num.bonus > p > span').text
print(title2, bonus_number)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 3


In [39]:
# 위의 select 함수를 find함수로
#times = soup.select_one('div.win_result > h4 > strong').text # 1173회
win_result = soup.find('div', class_="win_result")
times = win_result.find('strong').text
# date = soup.select_one('div.win_result >p.desc').text  #(2025년 05월 24일 추첨)
date = soup.find('p', class_='desc').text
print(times, date)
# title1 = soup.select_one('div.num.win > strong').text
num_win = soup.find('div', class_=['win'])
title1 = num_win.find('strong').text
# lotto_number = soup.select('div.num.win > p > span')
lotto_number = num_win.find_all('span') # 배열
print(title1, [int(lotto.text) for lotto in lotto_number])
# title2 = soup.select_one('div.num.bonus > strong').string
num_bonus = soup.find('div', class_=['bonus'])
title2 = num_bonus.find('strong').text
# bonus_number = soup.select_one('div.num.bonus > p > span').text
bonus_number = num_bonus.find('span').text
print(title2, bonus_number)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 3


### 3) 다음 검색 리스트
```
no  title   link
0 [비트코인 2025] 백악관 크립토 차르~ https://v.daum.net/v/20250528103907230
1 [비즈 나우] 비트코인 2025 컨퍼런~   https://v.daum.net/v/20250528075215864
....
```

In [56]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                           'title': item.text,
                           'link': item.attrs['href']})
import pandas as pd
pd.DataFrame(items_find_list)

,no,title,link
0,0,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
1,1,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
2,2,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
3,3,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
4,4,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
5,5,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
6,6,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
7,7,"""비트코인 산다""…트럼프家, 25억달러 자금 조달 추진[코인브리핑]",http://v.daum.net/v/20250528111837527
8,8,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [64]:
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 2차원 리스트
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])
# items_find_list
import pandas as pd
pd.DataFrame(items_find_list, columns=['no', 'title', 'link'])

,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
2,2,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
3,3,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
4,4,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
5,5,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
8,8,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [77]:
# 다음 뉴스 검색(키워드, 원하는 페이지수)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
keyword = '비트코인'
page = 2
# url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&q={keyword}&p={page}'
# response = requests.get(url)
url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
params = {'q':keyword, 'p':page}
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    print({'no': (page-1)*10 + idx,
           'title': item.text,
           'link': item.attrs['href']})

{'no': 10, 'title': ' 트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고 ', 'link': 'http://v.daum.net/v/20250528084245020'}
{'no': 11, 'title': ' 연일 최고가 경신하는 비트코인…투자자들 강 건너 불구경하는 이유는 ', 'link': 'http://v.daum.net/v/20250526212400275'}
{'no': 12, 'title': ' "비트코인 산다"…트럼프家, 25억달러 자금 조달 추진[코인브리핑] ', 'link': 'http://v.daum.net/v/20250528111837527'}
{'no': 13, 'title': ' [김대호 박사의 오늘의 키워드] 소비자신뢰지수·비트코인 2025·머스크 한마디·애플 괘씸죄 ', 'link': 'http://v.daum.net/v/20250528071325361'}
{'no': 14, 'title': ' 잘 나가는 비트코인 산다는데 왜?…트럼프 미디어 주가 10% 급락 [투자360] ', 'link': 'http://v.daum.net/v/20250528074002668'}
{'no': 15, 'title': ' "테슬라 팔고 갈아탔어요"…자녀계좌 보던 엄마들 \'돌변\' [마켓PRO] ', 'link': 'http://v.daum.net/v/20250528094602647'}
{'no': 16, 'title': ' 달러 스테이블코인, 국내 거래 확산…당국 외환흐름 통제력 약화 우려 ', 'link': 'http://v.daum.net/v/20250528152520516'}
{'no': 17, 'title': ' 스트래티지, 비트코인 4020개 추가 매입...보유량 58만개 돌파 ', 'link': 'http://v.daum.net/v/20250527081116129'}
{'no': 18, 'title': ' 비트코인과 엇갈리는 이더리움·리플, 왜? ', 'link': 'http://v.daum.net/v/20

In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
    items_find_list = [] # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    return items_find_list

In [102]:
collect_list('청바지',2)

[{'no': 10,
  'title': ' 60대인데요, 청바지를 다시 입었습니다 ',
  'link': 'http://v.daum.net/v/20250430145405823'},
 {'no': 11,
  'title': " '45kg' 윤은혜, 청바지가 헐렁…'40세' 비현실적 자기관리 끝판왕 ",
  'link': 'http://v.daum.net/v/20250513174826265'},
 {'no': 12,
  'title': ' 173㎝ 장원영 맵시 美쳤네, 청바지 하나 입었을 뿐인데 [스타화보] ',
  'link': 'http://v.daum.net/v/20250522115656470'},
 {'no': 13,
  'title': ' 손담비, 출산 한 달도 안 됐는데 외출…청바지 폼 애엄마 맞나 ',
  'link': 'http://v.daum.net/v/20250506171902866'},
 {'no': 14,
  'title': ' 48세 김희선, 흰 셔츠+청바지 기본템 입었을 뿐인데‥미모 레전드 ',
  'link': 'http://v.daum.net/v/20250511134648392'},
 {'no': 15,
  'title': ' 윤은혜, 40세 맞아?…청바지도 헐렁한 극세사 몸매+동안 미모 ',
  'link': 'http://v.daum.net/v/20250515063005066'},
 {'no': 16,
  'title': ' “브래지어에 청바지만”…20년만에 속옷 모델 된 추성훈 아내 야노 시호 ‘파격’ ',
  'link': 'http://v.daum.net/v/20250515195400237'},
 {'no': 17,
  'title': ' ‘청바지’ 입은 안철수 ‘학식’ 이준석 쫓아갔지만 ',
  'link': 'http://v.daum.net/v/20250521160401791'},
 {'no': 18,
  'title': " 길은혜,'청바지 핏 뽐내며 입장' [사진] ",
  'link': 'http://v.daum.ne

In [103]:
# 원하는 keyword로 다음검색
result = []
keyword = "청바지"
pages = 4
for page in range(1, pages+1):
    result.extend(collect_list(keyword, page))
    time.sleep(3)
result_df = pd.DataFrame(result)
result_df.head()

,no,title,link
0,0,"'이병헌♥' 이민정, 매장 통째로 빌려 청바지 쇼핑",http://v.daum.net/v/20250528050507783
1,1,"“추앙받을 만 하네”… 김지원, 셔츠에 청바지만 입어도 공항 ‘정지’",http://v.daum.net/v/20250526182411395
2,2,흰 티셔츠에 청바지 정석대로 입는 법,http://v.daum.net/v/20250527185425864
3,3,"이민정, 매장 빌려 폭풍 쇼핑 ""10년 전 샀던 청바지는 응급실""(MJ)",http://v.daum.net/v/20250527191015309
4,4,"‘이병헌♥’ 이민정, 청바지 사는데 매장 통째로 빌렸다..남다른 쇼핑 클래스 “나...",http://v.daum.net/v/20250527192310643


In [122]:
keywords = ['청바지','면바지']
pages = 2
result0 = [] # 청바지 검색결과 50개
result1 = [] # 면바지 검색결과 50개
for i, keyword in enumerate(keywords):
    for page in range(1, pages+1):
        print(f'~ ~ ~ {i}번째 {keyword} {page} 검색 중입니다 ~ ~ ~')
        if i==0:
            result0.extend(collect_list(keyword, page))
        else:
            result1.extend(collect_list(keyword, page))
        time.sleep(4)

~ ~ ~ 0번째 청바지 1 검색 중입니다 ~ ~ ~
~ ~ ~ 0번째 청바지 2 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 1 검색 중입니다 ~ ~ ~
~ ~ ~ 1번째 면바지 2 검색 중입니다 ~ ~ ~


In [123]:
result0_df = pd.DataFrame(result0)
result0_df.head()

,no,title,link
0,0,"'이병헌♥' 이민정, 매장 통째로 빌려 청바지 쇼핑",http://v.daum.net/v/20250528050507783
1,1,"“추앙받을 만 하네”… 김지원, 셔츠에 청바지만 입어도 공항 ‘정지’",http://v.daum.net/v/20250526182411395
2,2,흰 티셔츠에 청바지 정석대로 입는 법,http://v.daum.net/v/20250527185425864
3,3,"이민정, 매장 빌려 폭풍 쇼핑 ""10년 전 샀던 청바지는 응급실""(MJ)",http://v.daum.net/v/20250527191015309
4,4,"‘이병헌♥’ 이민정, 청바지 사는데 매장 통째로 빌렸다..남다른 쇼핑 클래스 “나...",http://v.daum.net/v/20250527192310643


In [124]:
result1_df = pd.DataFrame(result1)
result1_df.head()

,no,title,link
0,0,사천에서 쾌청한 날씨 속 탁 트인 남해 풍경 즐겨요!,http://v.daum.net/v/20250528104429529
1,1,"윤승아, ♥김무열 쏙 빼닮은 깜찍 子 하객룩 공개‥벌써 우월한 유전자",http://v.daum.net/v/20250526145941690
2,2,"이번 주 포천 여행,반팔 하나로 충분할까?",http://v.daum.net/v/20250527171732518
3,3,"이재명, 첫 대학 방문…이준석 겨냥?",http://v.daum.net/v/20250526191942751
4,4,"""나는 젊어"" 팔소매 접은 이준석의 '흰셔츠' 속에도 뼈가 담겼다[21대 대선 리...",http://v.daum.net/v/20250525070013619


In [125]:
result0_df.to_csv('data/ch14_'+keywords[0]+'.csv', index=False, encoding='cp949')
result1_df.to_csv('data/ch14_'+keywords[1]+'.csv', index=False)

### 4) User-agent를 추가하여 크롤링
- urlopen()함수를 사용하면 크롤링이 안 되는 사이트
- User-agent를 추가하여 크롤링

In [121]:
# 방법2
from urllib.request import urlopen, Request
import urllib.parse
word = '비트코인'
word = urllib.parse.quote(word)
# print(word)
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
# User-Agent를 추가하여 브라우저처럼 보이게 포장
headers = {'user-agent':
         'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
response = urlopen(request)
soup = BeautifulSoup(response, # response.content
                    "html.parser")
def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
    items_find_list = [] # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    return items_find_list
collect_list("청바지", 1)

[{'no': 0,
  'title': " '이병헌♥' 이민정, 매장 통째로 빌려 청바지 쇼핑 ",
  'link': 'http://v.daum.net/v/20250528050507783'},
 {'no': 1,
  'title': ' “추앙받을 만 하네”… 김지원, 셔츠에 청바지만 입어도 공항 ‘정지’ ',
  'link': 'http://v.daum.net/v/20250526182411395'},
 {'no': 2,
  'title': ' 흰 티셔츠에 청바지 정석대로 입는 법 ',
  'link': 'http://v.daum.net/v/20250527185425864'},
 {'no': 3,
  'title': ' 이민정, 매장 빌려 폭풍 쇼핑 "10년 전 샀던 청바지는 응급실"(MJ) ',
  'link': 'http://v.daum.net/v/20250527191015309'},
 {'no': 4,
  'title': ' ‘이병헌♥’ 이민정, 청바지 사는데 매장 통째로 빌렸다..남다른 쇼핑 클래스 “나는 즉흥적”(이민정 MJ) ',
  'link': 'http://v.daum.net/v/20250527192310643'},
 {'no': 5,
  'title': ' 이시영처럼 다리 길어보이고 싶어서 청바지 분석함 [입스타그램] ',
  'link': 'http://v.daum.net/v/20250519172231644'},
 {'no': 6,
  'title': ' \'의사 그만둔\' 고윤정, 흰 티+청바지..."정석 미녀" [★해시태그] ',
  'link': 'http://v.daum.net/v/20250522182007490'},
 {'no': 7,
  'title': ' "청렴이 최고" … 창원특례시 성산구 문화위생과, \'청바지 데이\' 추진 ',
  'link': 'http://v.daum.net/v/20250511091428626'},
 {'no': 8,
  'title': ' 패셔니스타 김지원, 청바지가 어울리는 여자랍니다! [포토] ',

- 꼭 User-Agent를 사용하여야 하는 경우 : https://www.melon.com/chart/

In [128]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://www.melon.com/chart/'
melonpage = requests.get(url)
melonpage.status_code

406

In [131]:
# 방법2
from urllib.request import urlopen, Request
# melonpage = urlopen(url) 에러남

In [135]:
# User-Agent 추가
headers = {'user-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}
# 방법2
request = Request(url, headers=headers)
melonpage = urlopen(request)
# 방법1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    "html.parser")
#soup

In [142]:
# 멜론 순위, 노래제목, 가수, 좋아요갯수
# 1위 | 너에게 닿기를 | 10cm (100위까지 크롤링)
title_els = soup.select('div.ellipsis.rank01 > span')
singer_els = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
len([title.text.strip() for title in title_els]), \
len([singer.text for singer in singer_els])
for idx, (title, singer) in enumerate(zip(title_els, singer_els)):
    print('{}위 | {} - {}'.format(idx+1,
                                 title.text.strip(),
                                 singer.text))

1위 | 너에게 닿기를 - 10CM
2위 | Never Ending Story - 아이유
3위 | Drowning - WOODZ
4위 | like JENNIE - 제니 (JENNIE)
5위 | 모르시나요(PROD.로코베리) - 조째즈
6위 | THUNDER - 세븐틴 (SEVENTEEN)
7위 | TOO BAD (feat. Anderson .Paak) - G-DRAGON
8위 | 네모의 꿈 - 아이유
9위 | Whiplash - aespa
10위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGON
11위 | 나는 반딧불 - 황가람
12위 | 어제보다 슬픈 오늘 - 우디 (Woody)
13위 | 오늘만 I LOVE YOU - BOYNEXTDOOR
14위 | REBEL HEART - IVE (아이브)
15위 | HAPPY - DAY6 (데이식스)
16위 | 빨간 운동화 - 아이유
17위 | Flower - 오반(OVAN)
18위 | APT. - 로제 (ROSÉ), Bruno Mars
19위 | 청춘만화 - 이무진
20위 | HOT - LE SSERAFIM (르세라핌)
21위 | 10월 4일 - 아이유
22위 | 한 페이지가 될 수 있게 - DAY6 (데이식스)
23위 | Last Scene (Feat. 원슈타인) - 아이유
24위 | HBD - 세븐틴 (SEVENTEEN)
25위 | MY LOVE(2025) - 이예은, 아샤트리, 전건호
26위 | 그날이 오면 - 투모로우바이투게더
27위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴
28위 | Fly Up - RIIZE
29위 | Welcome to the Show - DAY6 (데이식스)
30위 | 소나기 - 이클립스 (ECLIPSE)
31위 | 천상연 - 이창섭
32위 | Die With A Smile - Lady Gaga, Bruno Mars
33위 | toxic till the end - 로제 (ROSÉ)
34위 | HANDS UP - MEOVV (미야오)
35위 | Superno

### 5)네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수(3)만큼 검색한 결과 데이터프레임으로 출력

In [5]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
# print(url)
response = get(url)
# response.status_code
soup = BeautifulSoup(response.text, # response.content
                    'html.parser')

In [7]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
# 쳇지피티 -> %EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
keyword = quote(keyword) # url 인코딩된 문자열로 전환
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
# print(url)
response = urlopen(url)
soup = BeautifulSoup(response, # response.read()
                     'html.parser')

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0


In [22]:
# 페이징 포함(keyword를 pages수 만큼 검색한 결과를 데이터프레임에 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
keyword = '쳇지피티'
pages = 3
items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # print(url)
    # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
    response = requests.get(url)
    # print(response.status_code) 정상 접근 가능한지 여부
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('dt > a')
    # print([item.attrs['href'] for item in items])
    for item in items:
        items_list.append([item.text, item.attrs.get('href')])
#         items_list.append({
#                 'title':item.text,
#                 'link':item.attrs.get('href')})
# print(len(items_list), '개 데이터')
# print(items_list[:2])
df = pd.DataFrame(items_list, columns=['title', 'link'])
df

,title,link
0,쳇지피티 논문 출처,https://kin.naver.com/qna/detail.naver?d1id=4&...
1,쳇지피티 한도,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,쳇지피티 사용 확인 AI,https://kin.naver.com/qna/detail.naver?d1id=5&...
3,쳇지피티 이거 뭐예요?해킹 뭐그런거...,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,쳇지피티 ai 소라 질문이요,https://kin.naver.com/qna/detail.naver?d1id=1&...
5,쳇지피티 초기화,https://kin.naver.com/qna/detail.naver?d1id=1&...
6,학교 생기부 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...
7,제가 직접 쓴 자소서 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...
8,쳇지피티 질문입니다.,https://kin.naver.com/qna/detail.naver?d1id=1&...
9,쳇지피티 채팅 문제,https://kin.naver.com/qna/detail.naver?d1id=1&...


In [25]:
# 페이징 포함(keyword를 pages수 만큼 검색한 결과를 데이터프레임에 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
keyword = '쳇지피티'
pages = 3
items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dts = soup.find_all('dt')
    for dt in dts:
        item = dt.find('a')
        items_list.append([item.text, item.attrs.get('href')])
df = pd.DataFrame(items_list, columns=['title', 'link'])
df.head(2)

,title,link
0,쳇지피티 논문 출처,https://kin.naver.com/qna/detail.naver?d1id=4&...
1,쳇지피티 한도,https://kin.naver.com/qna/detail.naver?d1id=1&...


In [26]:
# 함수(keyword를 pages수 만큼 검색한 결과를 데이터프레임에 )
def get_items_data(keyword, pages):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
    for page in range(1, pages+1):
        url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
        # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        dts = soup.find_all('dt')
        for dt in dts:
            item = dt.find('a')
            items_list.append([item.text, item.attrs.get('href')])
    return pd.DataFrame(items_list, columns=['title', 'link'])

In [28]:
df = get_items_data('청바지',3)
df.head(2)

,title,link
0,영어로 청바지를 뭐라고하나요?,https://kin.naver.com/qna/detail.naver?d1id=11...
1,"전 남자인데, 누워서 청바지를 입으면...",https://kin.naver.com/qna/detail.naver?d1id=7&...


### 6)네이버 지식인으로 검색(open API 사용O)
- 특정 keyword를 특정데이터 수(30)만큼 검색한 결과 데이터프레임으로 출력
    - .env에 발급받은 키 저장 -> load (pip install python-dotenv)

In [32]:
# 환경변수를 가져오기
from dotenv import load_dotenv
import os
load_dotenv() # dotenv_path='.env' 기본값
# print(os.getenv('Client_ID'))
# print(os.getenv('Client_Secret'))

True

In [40]:
# 네이버 검색 API 예제 - 지식인 검색
import os
import sys
import urllib.request
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # XML 결과

headers = {'X-Naver-Client-Id': client_id,
           'X-Naver-Client-Secret':client_secret}
request = urllib.request.Request(url, headers=headers)
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    import json
    data = json.loads(response_body.decode('utf-8')) # json형태의 str을 딕셔너리
    print(type(data))
    #print(data['items'])
else:
    print("Error Code:" + rescode)

<class 'str'>
<class 'dict'>


In [58]:
# keyword를 display수만큼 지식in검색 결과를 데이터프레임에 (방법1 크롤링)
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
load_dotenv() 
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
keyword = "쳇지피티"
cnt = 30
url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
headers = {'X-Naver-Client-Id': client_id,
           'X-Naver-Client-Secret':client_secret}
response = requests.get(url, headers=headers)
# response.status_code
# items = json.loads(response.text)['items'] : json.loads() -방법1과 방법2
items = response.json()['items'] # 방법1에서만 가능한 함수
items_list = []
for item in items:
    items_list.append([
        item.get('title').replace('<b>','').replace('</b>',''), # item['title']
        item.get('link'),
        item.get('description').replace('<b>','').replace('</b>','')
    ])
df = pd.DataFrame(items_list, columns=['title','link','description'])
df
#print(df.loc[0, 'link'])

,title,link,description
0,쳇지피티 논문 출처,https://kin.naver.com/qna/detail.naver?d1id=4&...,"쳇지피티한테 어떤 내용에 관한 논문을 요구하고, 해당... 지피티가 스스로 여러 논..."
1,쳇지피티 한도,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료로 쓰고 있는데 쳇지피티로 소설 하나 쓰고 있거든요 근데 한도가 초과됐...
2,쳇지피티 사용 확인 AI,https://kin.naver.com/qna/detail.naver?d1id=5&...,쳇지피티 사용 여부 확인하는 무료 AI 뭐가 있나요?? 좀 정확하고 확실한거로 알려...
3,쳇지피티 이거 뭐예요?해킹 뭐그런거 당하는거 아니죠?,https://kin.naver.com/qna/detail.naver?d1id=1&...,아니 제가 크롬을 쓰는데 자꾸 쳇지피티가 한글번역도 이상하게 되고 말도 끊기는 거예...
4,쳇지피티 ai 소라 질문이요,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 플러스 이용중이구요 어제까지만 해도 왼쪽 메뉴에 '소라' 가 있어서요 해서...
5,쳇지피티 초기화,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 대화 목록이 사라진건 아니고 맥락에 안 맞게 말하길래 한도 초과... 챗 ...
6,학교 생기부 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...,학교에서 생기부를 작성할 때 쳇지피티를 그대로 배낀 보고서를 내면 대학에서 잡는다는...
7,제가 직접 쓴 자소서 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...,제가 직접 제 이야기로 자소서를 쓰고 쳇지피티한테 보내고 지피티가 수정해준 내용 참...
8,쳇지피티 질문입니다.,https://kin.naver.com/qna/detail.naver?d1id=1&...,요즘 쳇지피티가 유행이길래 한번 깔아봤는데 인사조차 못알아먹는데 어떻게 하나요..?...
9,쳇지피티 채팅 문제,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티3.0 무료버전을 종종 사용합니다 제가 질문을 &quot;이번에 전시회를 할...


In [2]:
# 함수 작성
def get_naver_kin(keyword, cnt):
    import requests
    import pandas as pd
    import json
    from dotenv import load_dotenv
    import os
    load_dotenv() 
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')

    url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
    headers = {'X-Naver-Client-Id': client_id,
               'X-Naver-Client-Secret':client_secret}
    response = requests.get(url, headers=headers)

    # items = json.loads(response.text)['items'] : json.loads() -방법1과 방법2
    items = response.json()['items'] # 방법1에서만 가능한 함수
    items_list = []
    for item in items:
        items_list.append([
            item.get('title').replace('<b>','').replace('</b>',''), # item['title']
            item.get('link'),
            item.get('description').replace('<b>','').replace('</b>','')
        ])
    return pd.DataFrame(items_list, columns=['title','link','description'])

In [3]:
get_naver_kin("청바지",20).head()

,title,link,description
0,영어로 청바지를 뭐라고하나요?,https://kin.naver.com/qna/detail.naver?d1id=11...,영어로 청바지를 뭐라고하나요? 안녕하세요~ 선생님도 매월 시험보는 목동영어학원 길벗...
1,"전 남자인데, 누워서 청바지를 입으면 고환이 아플 수?",https://kin.naver.com/qna/detail.naver?d1id=7&...,"전 남자인데, 누워서 청바지를 입으면 고환이 아플 수? 안녕하세요. 하이닥-네이버 ..."
2,청바지 없는데 면접취소 할까요?,https://kin.naver.com/qna/detail.naver?d1id=6&...,낼 알바 면접인데 츄리닝만 있고 청바지가 없어요 취소할까요? 아님 청바지는 아니어도...
3,고관절 지방이식 후 청바지,https://kin.naver.com/qna/detail.naver?d1id=7&...,"현재 고관절(골반) 지방이식 한 지 두달 가까이 되어가는데,꽉 끼는 청바지나 레깅스..."
4,임영웅 청바지 입은 사진?,https://kin.naver.com/qna/detail.naver?d1id=3&...,임영웅 사진첩 만들고 있는데요! 청바지 입은 사진 갖고 계신분 공유부탁드려요! 임영...


### quiz) 네이버 open API를 이용해서 청바지 이미지 100건 데이터를 csv파일로 저장
        데이터 : 제목, 이미지링크, 썸네일링크,높이,폭

In [53]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv() 
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
word = "청바지"
url = f'https://openapi.naver.com/v1/search/image?query={word}&display=100'
headers = {'X-Naver-Client-Id': client_id,
           'X-Naver-Client-Secret':client_secret}
response = requests.get(url, headers=headers)
# print(response.text)
items = response.json()['items'] # 방법1에서만 가능한 함수
items_list = []
for idx, item in enumerate(items):
    title = item.get('title')
    link = item.get('link')
    thumbnail = item.get('thumbnail')
    sizeheight = item.get('sizeheight')
    sizewidth = item.get('sizewidth')
    items_list.append({
        'no': idx+1,
        'title': title,
        'link' : link,
        'thumbnail' : thumbnail,
        'sizeheight':sizeheight,
        'sizewidth' :sizewidth
    })
    # link와 thumbnail을 저장
df = pd.DataFrame(items_list)
df.head(1)

,no,title,link,thumbnail,sizeheight,sizewidth
0,1,BINIWU 심플 빈티지 워싱 청바지 남자 봄가을 루즈핏 통바지 28 빈티지블루,http://shopping.phinf.naver.net/main_5172227/5...,https://search.pstatic.net/common/?type=b150&s...,800,800


In [16]:
df.loc[3, 'link']

'https://thumbnail8.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/images/2017/09/07/17/1/837feb8c-041e-4921-bd39-174fec765a2a.jpg'

In [20]:
df[df['link'].str.find('?')!=-1]

,no,title,link,thumbnail,sizeheight,sizewidth


In [28]:
df[~df['link'].str.endswith('jpg')].loc[5, 'link']

'http://shop1.phinf.naver.net/20250503_295/1746250611381Vinxl_JPEG/21192829047529977_2039968073.jpeg'

In [6]:
def save_img(attr, idx, query, url):
    import requests
    '{attr}_{idx}_{query}.{확장자} 이미지 저장'
    file_extension = url.split('.')[-1]
    quote_index = file_extension.find('?')
    
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    if len(file_extension) > 10:
        file_extension = 'jpg'
#     print(file_extension)
    img = requests.get(url).content
    open(f'ch14_image/{attr}_{idx}_{query}.{file_extension}', 'wb').write(img)

In [60]:
save_img('메인', 1, '청바지', df.loc[5, 'link'])

In [9]:
# word image 100개 검색한 데이터는 csv파일로, 이미지는 ch14_image폴더에 저장되는 함수
def get_naver_save_image(word):
    import requests
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv() 
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')

    url = f'https://openapi.naver.com/v1/search/image?query={word}&display=100'
    headers = {'X-Naver-Client-Id': client_id,
               'X-Naver-Client-Secret':client_secret}
    response = requests.get(url, headers=headers)

    items = response.json()['items'] # 방법1에서만 가능한 함수
    items_list = []
    for idx, item in enumerate(items):
        title = item.get('title')
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        sizeheight = item.get('sizeheight')
        sizewidth = item.get('sizewidth')
        items_list.append({
            'no': idx+1,
            'title': title,
            'link' : link,
            'thumbnail' : thumbnail,
            'sizeheight':sizeheight,
            'sizewidth' :sizewidth
        })
        # link와 thumbnail을 저장
        save_img('메인', idx+1, word, link)
        save_img('썸네일', idx+1, word, thumbnail)
        # 20%, 40%, 60% 80% 지점 message 출력
        if idx/20 == round(idx/20):
            print(f'= = = {idx}% 진행 중입니다 = = =')
    df = pd.DataFrame(items_list)
    df.to_csv(f'ch14_image/{word}.csv',  index=False)
    print('~ ~ ~ 이미지 및 csv 파일 저장 완료 ~ ~ ~')

In [10]:
get_naver_save_image('어린이집')

= = = 0% 진행 중입니다 = = =
= = = 20% 진행 중입니다 = = =
= = = 40% 진행 중입니다 = = =
= = = 60% 진행 중입니다 = = =
= = = 80% 진행 중입니다 = = =
~ ~ ~ 이미지 및 csv 파일 저장 완료 ~ ~ ~


## 2.3 xml 파일
- 기상예측 RSS 활용
- http://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml

In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml'
response = urlopen(url)
# response.status # response.getcode()
soup = BeautifulSoup(response, "xml") # pip install lxml
locals = soup.find_all('local_ta')
# locals[1]
items_list = []
for local in locals:
    local_ta_name = local.select_one('local_ta_name').text
    week1_normalYear = local.select_one('week1_local_ta_normalYear').text #평년기온
    week1_similarRange = local.select_one('week1_local_ta_similarRange').text # 기온범위
    week1_min = local.select_one('week1_local_ta_minVal').text
    week1_max = local.select_one('week1_local_ta_maxVal').text
    week2_normalYear = local.select_one('week2_local_ta_normalYear').text #평년기온
    week2_similarRange = local.select_one('week2_local_ta_similarRange').text # 기온범위
    week2_min = local.select_one('week2_local_ta_minVal').text
    week2_max = local.select_one('week2_local_ta_maxVal').text
    items_list.append({
        '지역':local_ta_name,
        '1주평년기온':float(week1_normalYear),
        '1주범위':week1_similarRange,
        '1주최저':week1_min,
        '1주최고':week1_max,
        '2주평년기온':week2_normalYear,
        '2주범위':week2_similarRange,
        '2주최저':week2_min,
        '2주최고':week2_max
    })
df = pd.DataFrame(items_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      13 non-null     object 
 1   1주평년기온  13 non-null     float64
 2   1주범위    13 non-null     object 
 3   1주최저    13 non-null     object 
 4   1주최고    13 non-null     object 
 5   2주평년기온  13 non-null     object 
 6   2주범위    13 non-null     object 
 7   2주최저    13 non-null     object 
 8   2주최고    13 non-null     object 
dtypes: float64(1), object(8)
memory usage: 1.0+ KB


# 3. 연습문제
- YES24 베스트셀러 정보를 ch14_yes24.csv(ch14_yes24.txt)로 출력
- 순위, 책이름, 저자, 출판사, 가격 (1~48위) => 리스트 => 데이터프레임 => csv
                                                   => txt

In [32]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [64]:
# 방법1
pages = 2
bestseller_list = []
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    response = requests.get(url)
    # print(url)
    # print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks = [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.item_info > div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    writers_els = soup.select("span.authPub.info_auth")
    # writers_els = soup.find_all('span', class_='info_auth')
    writers = [writers_el.text.strip() for writers_el in writers_els]
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    for rank, title, writer, publisher, price in zip(ranks, titles, writers, publishers, prices):
        # print("{},{},{},{},{}".format(rank, title, writer, publisher, price))
        bestseller_list.append([rank, title, writer, publisher, price])
df = pd.DataFrame(bestseller_list, 
                  columns=['rank', 'title', 'writer', 'publisher', 'price'])
df.to_csv('data/ch14_yes24.csv', index=False)

In [65]:
# 방법2
pages = 2
bestseller_list = []
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    # print(url)
    response = urlopen(url)
    #print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks = [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.item_info > div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    writers_els = soup.select("span.authPub.info_auth")
    # writers_els = soup.find_all('span', class_='info_auth')
    writers = [writers_el.text.strip() for writers_el in writers_els]
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    for rank, title, writer, publisher, price in zip(ranks, titles, writers, publishers, prices):
        # print("{},{},{},{},{}".format(rank, title, writer, publisher, price))
        bestseller_list.append({
            'rank':rank, 
            'title':title,
            'writer':writer,
            'publisher':publisher,
            'price':price})
df = pd.DataFrame(bestseller_list, 
                  columns=['rank', 'title', 'writer', 'publisher', 'price'])
df.to_csv('data/ch14_yes24.csv', index=False)

200
200
